In [1]:
from sklearn.mixture import GaussianMixture
import sklearn.datasets as datasets
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as la
from scipy.stats import bernoulli

In [2]:
data = datasets.load_digits()
X = data['data']
y = data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

gmm = GaussianMixture(10, max_iter=500000)
gmm = gmm.fit(X_train)
# samples = gmm.sample(10)

# for i in range(10):
#   plt.imshow(samples[0][i].reshape((8,8)), cmap='gray')
#   plt.title(samples[1][i])
#   plt.show()

# for i in range(10):
#   digit = gmm.means_[i]
#   plt.imshow(digit.reshape((8,8)), cmap='gray')
#   plt.title(f'{i}')
#   plt.show()

In [3]:
def acceptance(x, y, mu, sigma):
  p = min(1, np.exp(-0.5 * (np.dot(x - mu, la.solve(
    sigma, x - mu)) - np.dot(y - mu, la.solve(sigma, y - mu)))))
  return bernoulli.rvs(p)


def nextState(x, mu, sigma):
  K = len(x)
  y = np.random.multivariate_normal(x, np.eye(K))
  accept = acceptance(y, x, mu, sigma)
  if accept:
    return y
  else:
    return x


def lmvnorm(x, mu, sigma):
  return -0.5 * (np.dot(x - mu, la.solve(sigma, x - mu)) - len(x) * np.log(2 * np.pi) - np.log(la.det(sigma)))


def metropolis(x, mu, sigma, n_samples=1000):
  logprobs = np.zeros(n_samples)
  x_samples = np.zeros((n_samples, len(x)))
  for i in range(n_samples):
    logprobs[i] = lmvnorm(x, mu, sigma)
    x = nextState(x, mu, sigma)
    x_samples[i, :] = x.copy()
  return x_samples, logprobs


In [ ]:
metropolis()